## Part 1
#### 1. Read the case.csv file from the 311 call data into a Spark DataFrame.
#### 2. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
#### 3. How many Stray Animal cases are there?
#### 4. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
#### 5. Create a new DataFrame without any information related to dates or location.
#### 6. Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [1]:
# set up environment for spark
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

#### 1. Read the case.csv file from the 311 call data into a Spark DataFrame.

In [2]:
# start spark session
spark = SparkSession.builder.getOrCreate()

In [3]:
# read in the csv into spark dataframe
df = (spark.read.csv('./sa311/case.csv', header=True, inferSchema=True))

#### 2. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [4]:
# check out our schema
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [5]:
# import pyspark functions
from pyspark.sql.functions import col, expr
import pyspark.sql.functions as F

In [6]:
# 2. How old is the latest (in terms of days past SLA) currently open issue? 
print('{:.4} days past SLA'.format(df.select('case_closed', 'num_days_late', 
          F.to_timestamp(df.case_opened_date, 'M/d/y H:mm').alias('timestamp'))\
.select('num_days_late', 'timestamp', F.datediff(F.current_timestamp(), col('timestamp')).alias('days_old'))\
.where(df.case_closed == 'NO')\
.sort('num_days_late', ascending=False)\
.first()['num_days_late']))

348.6 days past SLA


In [7]:
# How long has the oldest (in terms of days since opened) currently opened issue been open?

In [8]:
print(str(df.select('case_closed', 'num_days_late', 
          F.to_timestamp(df.case_opened_date, 'M/d/y H:mm').alias('timestamp'))\
.select('num_days_late', 'timestamp', F.datediff(F.current_timestamp(), col('timestamp')).alias('days_old'))\
.where(df.case_closed == 'NO')\
.sort('days_old', ascending=False)\
.first()['days_old']) + ' days since opened')

865 days since opened


#### 3. How many Stray Animal cases are there?

In [9]:
print(str(df.select('service_request_type')\
.where(df.service_request_type == 'Stray Animal').count())+' Stray Animal cases')

26760 Stray Animal cases


#### 4. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [10]:
print(str(df\
.select('dept_division', 'service_request_type')\
.filter(df.service_request_type != 'Officer Standby')\
.where(df.dept_division == 'Field Operations').count())\
      + ' service requests assigned to Field Operations that are not Officer Standby Request Types')

113902 service requests assigned to Field Operations that are not Officer Standby Request Types


#### 5. Create a new DataFrame without any information related to dates or location.

In [11]:
df_no_dates_or_places = df.select('dept_division', 'service_request_type', 'case_status', 'source_id')

In [12]:
df_no_dates_or_places.show(5)

+----------------+--------------------+-----------+---------+
|   dept_division|service_request_type|case_status|source_id|
+----------------+--------------------+-----------+---------+
|Field Operations|        Stray Animal|     Closed| svcCRMLS|
|     Storm Water|Removal Of Obstru...|     Closed| svcCRMSS|
|     Storm Water|Removal Of Obstru...|     Closed| svcCRMSS|
|Code Enforcement|Front Or Side Yar...|     Closed| svcCRMSS|
|Field Operations|Animal Cruelty(Cr...|     Closed| svcCRMSS|
+----------------+--------------------+-----------+---------+
only showing top 5 rows



#### 6. Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [13]:
df = (spark.read.csv('./sa311/dept.csv', header=True, inferSchema=True))

In [14]:
df = df.na.fill({'dept_name': 'Other'})

In [15]:
df.select(df.dept_name).where((df.dept_name == 'Other')).show()

+---------+
|dept_name|
+---------+
|    Other|
+---------+



### Part 2
#### 1. Convert the council_district column to a string column.
#### 2. Extract the year from the case_closed_date column.
#### 3. Convert num_days_late from days to hours in new columns num_hours_late.
#### 4. Convert the case_late column to a boolean column.
#### 5. Convert the SLA_days columns to a double column. Pull it all together

#### 1. Convert the council_district column to a string column.

In [16]:
df = (spark.read.csv('./sa311/case.csv', header=True, inferSchema=True))
df.select("council_district", F.format_string("%010d", "council_district").\
          alias("council_district_fixed")).show(5)

+----------------+----------------------+
|council_district|council_district_fixed|
+----------------+----------------------+
|               5|            0000000005|
|               3|            0000000003|
|               3|            0000000003|
|               3|            0000000003|
|               7|            0000000007|
+----------------+----------------------+
only showing top 5 rows



#### 2. Extract the year from the case_closed_date column.

In [17]:
df.select('case_closed_date').show(1)

+----------------+
|case_closed_date|
+----------------+
|    1/1/18 12:29|
+----------------+
only showing top 1 row



In [18]:
year_re = r'^.+/.+/(.+)\s+(.+)$'
df.select(F.regexp_extract(df.case_closed_date, year_re, 1).alias('year')).show()


+----+
|year|
+----+
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
|  18|
+----+
only showing top 20 rows



#### 3. Convert num_days_late from days to hours in new columns num_hours_late.

In [19]:
df.select("num_days_late", F.round(df.num_days_late * 24.00, 2).alias("num_hours_late")) \
  .show(5)

+-------------------+--------------+
|      num_days_late|num_hours_late|
+-------------------+--------------+
| -998.5087616000001|     -23964.21|
|-2.0126041669999997|         -48.3|
|       -3.022337963|        -72.54|
|       -15.01148148|       -360.28|
|0.37216435200000003|          8.93|
+-------------------+--------------+
only showing top 5 rows



#### 4. Convert the case_late column to a boolean column.

In [20]:
df.select(df.case_late == 'YES').show(10)

+-----------------+
|(case_late = YES)|
+-----------------+
|            false|
|            false|
|            false|
|            false|
|             true|
|            false|
|            false|
|            false|
|            false|
|            false|
+-----------------+
only showing top 10 rows



#### 5. Convert the SLA_days columns to a double column. Pull it all together

In [21]:
df.select(F.rint(df.SLA_days)).show(5)

+---------------+
|ROUND(SLA_days)|
+---------------+
|          999.0|
|            4.0|
|            4.0|
|           16.0|
|            0.0|
+---------------+
only showing top 5 rows



## Part 3
#### 1. Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).
#### 2. Join the case data with the source and department data.
#### 3. Are there any cases that do not have a request source?

#### 1. Create a DataFrame with all combinations of council_district and service_request_type (regardless of whether the combination is observed in the data).

In [59]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int, source_id: string, source_username: string, dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]>

In [27]:
df1 = df.select('council_district')
df2 = df.select('service_request_type')
df1.crossJoin(df2).show(5)

+----------------+--------------------+
|council_district|service_request_type|
+----------------+--------------------+
|               5|        Stray Animal|
|               3|        Stray Animal|
|               3|        Stray Animal|
|               3|        Stray Animal|
|               7|        Stray Animal|
+----------------+--------------------+
only showing top 5 rows



#### 2. Join the case data with the source and department data.

In [82]:
df_cases = spark.read.csv('./sa311/case.csv', header=True, inferSchema=True)
df_source = spark.read.csv('./sa311/source.csv', header=True, inferSchema=True)
df_dept = spark.read.csv('./sa311/dept.csv', header=True, inferSchema=True)

In [83]:
df_cases.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [84]:
df_source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [85]:
df_dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [86]:
df = df_cases.join(df_dept, df_cases.dept_division == df_dept.dept_division, 'left')

In [87]:
df = df.join(df_source, df.source_id == df_source.source_id, "left")

In [88]:
df.limit(10).toPandas().head()

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,case_status,source_id,request_address,council_district,dept_division,dept_name,standardized_dept_name,dept_subject_to_SLA,source_id,source_username
0,1014127332,1/1/18 0:42,1/1/18 12:29,9/26/20 0:42,NO,-998.508762,YES,Field Operations,Stray Animal,999.000000,Closed,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5,Field Operations,Animal Care Services,Animal Care Services,YES,svcCRMLS,svcCRMLS
1,1014127333,1/1/18 0:46,1/3/18 8:11,1/5/18 8:30,NO,-2.012604,YES,Storm Water,Removal Of Obstruction,4.322222,Closed,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3,Storm Water,Trans & Cap Improvements,Trans & Cap Improvements,YES,svcCRMSS,svcCRMSS
2,1014127334,1/1/18 0:48,1/2/18 7:57,1/5/18 8:30,NO,-3.022338,YES,Storm Water,Removal Of Obstruction,4.320729,Closed,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3,Storm Water,Trans & Cap Improvements,Trans & Cap Improvements,YES,svcCRMSS,svcCRMSS
3,1014127335,1/1/18 1:29,1/2/18 8:13,1/17/18 8:30,NO,-15.011481,YES,Code Enforcement,Front Or Side Yard Parking,16.291887,Closed,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3,Code Enforcement,Code Enforcement Services,DSD/Code Enforcement,YES,svcCRMSS,svcCRMSS
4,1014127336,1/1/18 1:34,1/1/18 13:29,1/1/18 4:34,YES,0.372164,YES,Field Operations,Animal Cruelty(Critical),0.125000,Closed,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7,Field Operations,Animal Care Services,Animal Care Services,YES,svcCRMSS,svcCRMSS


#### 3. Are there any cases that do not have a request source?

In [67]:
first_inst = df_cases.join(df_source, df_cases.source_id == df_source.source_id, "left_anti").first()

In [68]:
if first_inst:
    print('There is at least one case without a request source')
else:
    print('No instances of cases without a request source!')

No instances of cases without a request source!


## Part 4
#### 1. Who are the top 10 service request types in terms of number of requests?
#### 2. Who are the top 10 service request types in terms of average days late?
#### 3. Does number of days late depend on department?
#### 4. How do number of days late depend on department division and request type?

#### 1. Who are the top 10 service request types in terms of number of requests?

In [69]:
df_srt_count = df.groupBy('service_request_type').agg(F.count('*'))
df_srt_count.sort('count(1)', ascending=False).show(10)

+--------------------+--------+
|service_request_type|count(1)|
+--------------------+--------+
|           No Pickup|   89210|
|Overgrown Yard/Trash|   66403|
|        Bandit Signs|   32968|
|        Damaged Cart|   31163|
|Front Or Side Yar...|   28920|
|        Stray Animal|   27361|
|Aggressive Animal...|   25492|
|Cart Exchange Req...|   22608|
|Junk Vehicle On P...|   21649|
|     Pot Hole Repair|   20827|
+--------------------+--------+
only showing top 10 rows



In [70]:
df.crosstab("service_request_type", "dept_name").limit(10).toPandas()

,service_request_type_dept_name,Animal Care Services,City Council,Code Enforcement Services,Customer Service,Development Services,Metro Health,Parks and Recreation,Solid Waste Management,Trans & Cap Improvements,null
0,Masonry,0,0,0,0,0,0,447,0,0,0
1,Guard Rail/Guard Post Damage,0,0,0,0,0,0,0,0,817,0
2,Multi Tenant Exterior,0,0,84,0,0,0,0,0,0,0
3,CPS Energy Metal Poles,0,0,893,0,0,0,0,0,0,0
4,Graffiti Public Property,0,0,4104,0,0,0,0,0,0,0
5,Dead End/No Outlet Sign Request,0,0,0,0,0,0,0,0,155,0
6,"""Vacant Lot-Private (12"""")""",0,0,7875,0,0,0,0,0,0,0
7,"""Vacant Lot-State (48"""")""",0,0,15,0,0,0,0,0,0,0
8,3rd Call Bagged Leaves with Fee,0,0,0,0,0,0,0,24,0,0
9,Food Contact Surfaces,0,0,0,0,0,606,0,0,0,0


In [71]:
from pyspark.sql.types import DoubleType

df = df.withColumn("num_days_late", df["num_days_late"].cast(DoubleType()))\
        .withColumn("SLA_days", df["SLA_days"].cast(DoubleType()))\
        .withColumn('dept_lower', F.lower(F.col('dept_name')))

#### 2. Who are the top 10 service request types in terms of average days late?

In [72]:
df_srt_mdl = df.groupBy('service_request_type').agg(F.mean('num_days_late'))
df_srt_mdl.printSchema()
df_srt_mdl.sort('avg(num_days_late)', ascending=False).show(10)

root
 |-- service_request_type: string (nullable = true)
 |-- avg(num_days_late): double (nullable = true)

+--------------------+------------------+
|service_request_type|avg(num_days_late)|
+--------------------+------------------+
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
+--------------------+------------------+
only showing top 10 rows



#### 3. Does number of days late depend on department?

In [73]:
df_srt_mdl = df.groupBy('dept_lower').agg(F.mean('num_days_late'))
df_srt_mdl.printSchema()
df_srt_mdl.sort('avg(num_days_late)', ascending=False).show(20)

root
 |-- dept_lower: string (nullable = true)
 |-- avg(num_days_late): double (nullable = true)

+--------------------+-------------------+
|          dept_lower| avg(num_days_late)|
+--------------------+-------------------+
|                null| 135.92851612479797|
|    customer service| 59.737091496300785|
|development services|  13.43372455586971|
|solid waste manag...|-2.2000575136721747|
|        metro health| -4.911766979607002|
|parks and recreation| -5.251521960055133|
|trans & cap impro...|-20.612837354052626|
|code enforcement ...|-38.701330683295375|
|animal care services|-226.51783940550382|
|        city council|               null|
+--------------------+-------------------+



#### 4. How do number of days late depend on department division and request type?

In [51]:
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [92]:
df = df_dept.join(df_cases, on='dept_division')

In [105]:
df_gbdiv_req = df.groupby("dept_division", "service_request_type")\
        .mean('num_days_late')
df_gbdiv_req = df_gbdiv_req.sort('avg(num_days_late)', ascending=False)
df_gbdiv_req.show()

+--------------------+--------------------+------------------+
|       dept_division|service_request_type|avg(num_days_late)|
+--------------------+--------------------+------------------+
|    Code Enforcement|  Zoning: Junk Yards| 175.9563621042095|
|Code Enforcement ...|Labeling for Used...|162.43032902285717|
|Code Enforcement ...|Record Keeping of...|153.99724039428568|
|Code Enforcement ...|Signage Requied f...|151.63868055333333|
|Code Enforcement ...|Storage of Used M...|     142.112556415|
|Code Enforcement ...|Zoning: Recycle Yard|135.92851612479797|
|    Code Enforcement|Donation Containe...|131.75610506358706|
|Code Enforcement ...|License Requied U...|128.79828704142858|
|Code Enforcement ...|Used Mattress Ins...| 92.93570056835293|
|Code Enforcement ...|        Bandit Signs| 87.61764120320001|
|             Signals|Traffic Signal Gr...| 77.90021217000002|
|     311 Call Center|           Complaint| 72.51790932659713|
|    Code Enforcement|             Vendors| 66.11831254

In [1]:
spark.stop()

NameError: name 'spark' is not defined